# SNEAKSAVER IMAGE RECOGINATION TOOL

In [52]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers, losses


from keras.models import Sequential
import tensorflow_hub as hub
from tensorflow.keras.utils import to_categorical
from keras.layers import Conv2D, MaxPool2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Dropout, Dense, Flatten, Input, Lambda


from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau , EarlyStopping
from tensorflow.keras.optimizers import Adam 
import os

In [53]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath("")))

train_dir = os.path.join(BASE_DIR, "media", "dataset", "train")
test_dir = os.path.join(BASE_DIR, "media", "dataset", "test")


In [54]:
train_datagen = ImageDataGenerator(rescale = 1./ 255, rotation_range = 40, width_shift_range = 0.2, 
                                   height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, 
                                   horizontal_flip = True, fill_mode = 'nearest')

In [55]:
test_datagen = ImageDataGenerator(rescale = 1./ 255)

In [56]:
train_data = train_datagen.flow_from_directory(directory = train_dir, batch_size = 32, target_size = (240,240), class_mode = "categorical", shuffle = False)
test_data = test_datagen.flow_from_directory(directory = test_dir, batch_size = 32, target_size = (240,240), class_mode = "categorical")

Found 711 images belonging to 3 classes.
Found 114 images belonging to 3 classes.


In [46]:
model = tf.keras.models.Sequential([
    Conv2D(16, (3,3), activation = 'relu', input_shape = (240,240, 3)),
    MaxPooling2D(2,2),
    Conv2D(32, (3,3), activation = 'relu'),
    MaxPooling2D(2,2),
    Conv2D(32, (3,3), activation = 'relu'),
    MaxPooling2D(2,2),
    Conv2D(32, (3,3), activation = 'relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation = 'relu'),
    Dropout(0.2),
    Dense(3, activation = 'softmax')
])

In [47]:
# Using the Adam Optimizer to set the learning rate of our final model
opt = optimizers.Adam(learning_rate=0.0001)

# Compiling and setting the parameters we want our model to use
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy'])

In [48]:
for image_batch, label_batch in train_data:
  break
image_batch.shape, label_batch.shape

label_batch[0:5]
print (train_data.class_indices)

{'adidas': 0, 'converse': 1, 'nike': 2}


In [49]:
#write all the labels to a text file
labels = "\n".join(sorted(train_data.class_indices.keys()))
with open("labels.txt", "w") as f:
 f.write(labels)
!cat labels.txt

Access is denied.


In [50]:
history = model.fit(train_data, epochs = 10, steps_per_epoch = len(train_data), validation_data = test_data, validation_steps = int(0.25 * len(test_data)))

Epoch 1/10
23/23 [==============================] - 290s 11s/step - loss: 1.1461 - accuracy: 0.3038 - val_loss: 1.1098 - val_accuracy: 0.1875
Epoch 2/10
23/23 [==============================] - 205s 9s/step - loss: 1.1185 - accuracy: 0.2152 - val_loss: 1.0966 - val_accuracy: 0.4062
Epoch 3/10
23/23 [==============================] - 190s 8s/step - loss: 1.1082 - accuracy: 0.2672 - val_loss: 1.0958 - val_accuracy: 0.3750
Epoch 4/10
23/23 [==============================] - 292s 12s/step - loss: 1.1087 - accuracy: 0.3361 - val_loss: 1.0901 - val_accuracy: 0.2812
Epoch 5/10
 8/23 [=========>....................] - ETA: 2:47 - loss: 1.0940 - accuracy: 0.3984

KeyboardInterrupt: 

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 238, 238, 16)      448       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 119, 119, 16)      0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 117, 117, 32)      4640      
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 58, 58, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 56, 56, 32)        9248      
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 28, 28, 32)       

In [45]:
# Create a generator for test data
test_data_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    batch_size=32,
    target_size=(240, 240),
    class_mode="categorical",
    shuffle=False  # Ensure order is maintained for evaluation
)

# Evaluate the model using the generator
test_loss, test_accuracy = model.evaluate(test_data_generator)

print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

Found 114 images belonging to 3 classes.
4/4 [==============================] - 14s 2s/step - loss: 1.0708 - accuracy: 0.3333
Test Loss: 1.070845365524292
Test Accuracy: 0.3333333432674408


In [32]:
# Specify the path where you want to save the model
save_path = os.path.join(BASE_DIR, "sneakers_image_recog", "ml_model", 'sneak_image_recog_model.h5')

# Save the model to the specified path
model.save(save_path)

### Reference

https://www.kaggle.com/code/manarkandeel/sneakers-brand-classification/notebook

In [33]:
softmax_layer = model.layers[-1]

# Extract the class labels from the softmax layer
class_labels = softmax_layer.get_weights()[1]

# Print the class labels
print("Class Labels:", class_labels)

Class Labels: [ 4.0680476e-05 -2.8107778e-04 -5.3062500e-05]
